# Projekt 1 - Filmkészítés

Olvastál egy pályázati lehetőséget, ahol díjat adnak annak aki a legérdekesebb rövidfilmet készíti el. Mivel filmrendezési tapasztalataid és komoly kameráid sincsenek, arra gondoltál, hogy egy 'animált' rövidfilmet készíthetnél. Elővetted a jól elrejtett Lego kalóz figuráidat és kitaláltad, hogy leforgatod a Karib tenger kalózai c. film rövid Lego változatát. 

Szerencsére kéznél van a Raspberry Pi számítógép is, amire rá tudunk csatlakoztatni egy webkamerát. Rácsatlakoztatunk még egy nyomógombot is, amivel triggereljük majd a kép készítésének pillanatát. A gomb megnyomása előtt csak el kell helyezni a Lego figurákat a megfelelő pozícióba és elkészíteni róluk a képet. Majd elmozdítani a figurákat, amivel jelezzük a filmünk cselekményének fejlődését és egy újabb képet készítünk. Mindezt addig ismételjük, amíg vége nincs a cselekménysorozatnak. 

Az így készített képeket már csak egymás után kell vágni és egy .gif-et vagy .avi-t vagy .mp4-t készíteni belőle és indulhat a film lejátszása. 

## Mit fogsz készíteni?

Egy webkamerából (vagy Picam-ből) és egy nyomógombból álló rendszert készítünk, ahol a nyomógomb lenyomásával webkamera képet mentünk le a számítógépünkre. Ha nincs Raspberry Pi (vagy webkamera) a közelben akkor a laptopot és annak a beépített  kameráját és billentyűzetét is lehet használni erre a feladatra.

## Mit tanulsz meg?

A filmes projekt elkészítésével a következőket tanulod meg:

* Hogyan futtas terminál parancsokat Pythonból.
* Hogyan használd az ```opencv``` csomagot a webkamerával való kommunikálásra.
* Hogyan ments el képeket gombnyomásra.

## A projekt részletekre bontása

* Elkészíteni az áramkört.
* Létrehozni a kapcsolatot a webkamerával az ```opencv``` csomag használatával. 
* Inicializálni egy számlálót, ami számolja hány képet készítettünk.
* Végtelen ciklusban várni a gombnyomásra, ami triggereli a kép készítését.
* Lementeni a képet a harddiskre a számlálóval ellátott névvel.
* A ```q``` billentyű lenyomásával jelezni, hogy befejeztük.
* Bezárni a kapcsolatot a kamerával.
* Videó formátumba alakítani a képeket az ```ffmpeg``` paranccsal.

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek)  (vagy laptop)

b) [Webkamera](https://www.emag.hu/iuni-k6i-webkamera-full-hd-1080p-mikrofonnal-usb-2-0-plug-play-515422/pd/DX66N2MBM/?cmpid=87141&gclid=CjwKCAjwj6SEBhAOEiwAvFRuKL7E3Z6v7Ei_MNy1eFxoAn4ySFojVRVyiqf8BByR43dhONUlKDsrPBoC4sIQAvD_BwE) vagy [Picam](https://malnapc.hu/raspberry-pi-camera-board-v2-8mp)

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

d) [Nyomógomb](https://hu.farnell.com/schurter/1301-9320/switch-smd-push-12-5mm/dp/1217772?gclid=Cj0KCQjwzZj2BRDVARIsABs3l9K-ACTnuRr-dLVcDUKleNfECM3H3kWS_RfWtmXGMXVJeY9otda4dDcaAvGLEALw_wcB&gross_price=true&mckv=sSFnjERxR_dc|pcrid|434487710397|plid||kword||match||slid||product|1217772|pgrid|101346804139|ptaid|pla-389347076066|&CMP=KNC-GHU-SHOPPING-switches-relays-NEWSTRUCTURE-13-MAY-20) amit akár [építhetsz]() is.

## A kapcsolási rajz

<img src="schema/prog01_schema.png" width=600 height=400 />

1) A nyomógomb lábait illesszük a breadboard mélyedéseibe. Ügyeljünk arra, hogy a nyomógomb egy (vízszintes) sorba kerülő lábai a gomb lenyomása nélkül is kapcsolatban legyenek egymással, míg a (függőlegesen) egy sorba kerülő lábai csak a lenyomás következtében lesznek összekötve. Mindezt egy műszerrel leellenőrízhetjük.

2) Az egyik jumper kábelt (kék) kössük a gomb egyik lábának sorába, míg a másik végét a Raspberry Pi **GPIO 02**-es jelöléssel ellátott tüskéjére. Ez a tüske alapértelmezetten pull-up (magas vagy 1) módban van, azaz 3.3 V-ot mutat a földeléshez képest.

3) A másik jumper kábelt (narancssárga) kössük a gomb másik lábának sorába, míg a másik végét a Raspberry Pi **GND** (pl. **6**-os) jelöléssel ellátott tüskéjére. Ez a tüske alapértelmezetten földelés.

## A kód

Nyissunk meg egy új python fájlt és mentsük el pl. ```stop_motion.py``` név alatt. 

### A nyomógomb tesztelése

Első lépésként beimportáljuk a szükséges csomagokat:

* ```cv2``` - webkamerával való kommunikálásra az opencv csomag.
* ```gpiozero``` - a ```Button``` klasszt importáljuk be. 
* ```time``` - a ```sleep``` függvényt használjuk majd késleltetésre.
* ```subprocess``` - Evvel a csomaggal tudunk a terminál parancsokat végrehajtani pythonból.

Ezután inicializáljunk egy nyomógombot, ```button```, amit a *2*-es GPIO tüskékre kötünk. Most írjunk egy függvényt, ```pressed()```, ami a gomb lenyomására kiírja a képernyőre, hogy ```You pressed a button```. Ezt a függvényt össze is kell kötnünk a nyomógombbal, azaz hozzárendeljük a nyomógomb ```when_pressed``` metódusához. Majd írunk egy üres végtelen ```while``` ciklust, hogy a programunk ne álljon meg, amíg a gombot teszteljük.

```stop_motion.py```:

In [ ]:
import cv2, time, subprocess
from gpiozero import Button

button = Button(2)

def pressed():
    print('You pressed the button!')
    
button.when_pressed = pressed

while True:
    pass

### A kamera tesztelése

A következő lépés a kamera tesztelése. Ehhez létrehozzuk a kapcsolatot a kamerával, ```cap = cv2.VideoCapture(0)```. A zárójelben megadott szám a kamera indexére utal (0 az első számú kamera a rendszerben, de ha van több is akkor, 1-et, 2-őt stb. írva kiválaszthatjuk azokat is). Ha nem tudjuk a kameránk indexét, írjunk ```-1```-et. Linux alatt megeshet, hogy az indexelés nem működik, akkor nézzük, meg, az ```ls /dev/video*``` paranccsal a terminálban nézzük meg, hogy milyen kameráink vannak, mielőtt bedugnánk a kameránkat, majd ismét miután bedugtuk. Az újonnan megjelenő elem lesz a mi kameránk lokációja, és evvel is tudunk kapcsolatot teremteni pythonból, ```cap = cv2.VideoCapture('/dev/video0')```.

A ```cap.isOpened()``` paranccsal akár le is ellenőrizhetjük, hogy valóban megnyílt-e a kommunikációs csatorna a gép és a kamera között. Ha a válasz ```False```, akkor próbáljuk meg a ```cap.open()``` (Linux alatt a ```cap.open('/dev/video0')```) paranccsal megnyitni a kommunikációt. 

A ```cap.read()``` paranccsal tudunk fényképet készíteni, amely parancsnak két visszaadott értéke van, az első, ```ret``` megmondja, hogy a fénykép sikeresen elkészült-e, míg a második, ```frame``` tartalmazza a kép ```numpy.array``` mátrixát. Ezt a képet a ```cv2.imshow('frame', frame)``` paranccsal meg tudjuk jeleníteni (vagy használhatjuk a ```matplotlib.pyplot```-ból az ```imshow``` függvényt). 

Ezután várunk 3 másodpercet, ```time.sleep(3)```, mielőtt lezárnánk a kapcsolatot a kamerával, ```cap.release()```, amit érdemes minden egyes alkalommal megtenni, ha a programunknak vége van. A biztonság kedvéért még bezárunk minden ablakot is amit a programunk nyitott meg, ```cv2.destroyAllWindows()```.

```stop_motion.py```:

In [ ]:
import cv2, time, subprocess
from gpiozero import Button

cap = cv2.VideoCapture(0) # cap = cv2.VideoCapture('/dev/video0')
button = Button(2)

ret, frame = cap.read()
cv2.imshow('frame', frame)

time.sleep(3)

cap.release()
# Bezarunk minden ablakot, amit a program megnyitott
cv2.destroyAllWindows()

### ffmpeg parancs videók készítésére

Most áttekintjük, hogyan tudunk videót készíteni a fényképeinkből. A lenti példában 3 másodpercenként fogunk 4 db képet készíteni automatikusan, majd azokat filmmé alakítjuk. 

Az első dolog amit meg kell oldanunk, az a fényképek lementése, megfelelő név alá. Ehhez bevezetünk egy ```nframe``` változót, ami számolja, hogy hány fényképet készítettünk. Majd elindítunk egy ```for``` ciklust, ami 4-szer megy körbe. Minden egyes alkalommal készítünk egy képet, megnöveljük az ```nframe``` értékét és a ```cv2.imwrite(f'animation/frame{nframe:03d}.jpg', frame)``` paranccsal elmentjük a képet a jelenlegi mappánkban található ```animation``` mappába, *frame001.jpg*, *frame002.jpg* stb. néven. Egy példán keresztül bemutatjuk, hogy működik a számok megjelenítése:

In [1]:
nframe = 1
print(f'nframe = {nframe}')

nframe = 1


In [2]:
print(f'nframe = {nframe:03d}')

nframe = 001


A ```:03d``` a kapcsos zárójelben azt jelenti, hogy 3 jegyű számot szeretnénk string formájában megjeleníteni és ha a számunk nincs 3 jegyű, akkor az üres helyet töltse föl 0-val.

```stop_motion.py```:

In [ ]:
import cv2, time, subprocess
from gpiozero import Button

cap = cv2.VideoCapture(0) # cap = cv2.VideoCapture('/dev/video0')
button = Button(2)
nframe = 0

for i in range(4):
    ret, frame = cap.read()
    nframe = nframe + 1
    print(f'snaphot {nframe:03d} done!')
    cv2.imwrite(f'animation/frame{nframe:03d}.jpg', frame)
    time.sleep(3)

cap.release()
# Bezarunk minden ablakot, amit a program megnyitott
cv2.destroyAllWindows()

subprocess.call('ffmpeg -r 1 -i animation/frame%03d.jpg -qscale 1 animation.mp4')

Az elkészült képeket végül az ```ffmpeg``` paranccsal alakítjuk videóvá, amit pythonból hívunk meg a ```subprocess.call()``` függvényével. A ```-i``` argument után adhatjuk meg a képek nevét és lokációját. Mivel 4 képról van szó, a ```%03d``` kifejezés jelzi, hogy az össz képet vegye figyelembe ami *frame*-mel kezdődik és számokkal végződik. A ```-r``` argumentummal a frame rate-et lehet beállítani (másodpercben), hogy milyen gyorsan váltogassák a képek egymást. A ```-qscale``` pedig a videó minőségére vonatkozik. Végül a készítendő videó nevét adjuk meg, ```animation.mp4```.

### Filmkészítés Raspberry Pi-vel és nyomógombbal

A film elkészítéséhez feltételezzük, hogy nem tudjuk előre, hány képre lesz szükségünk, ezért egy végtelen ```while``` ciklust indítunk és inicializáljuk a képek számának tárolóját is, ```nframe```. A végtelen ciklus elindítása után időt hagyunk a Lego figurák elhelyezésére, ezért meghívjuk a ```button.wait_for_press()``` függvényt. Ha elkészültünk, megnyomhatjuk a gombot és elkészítjük a képet, megjelenítjük azt a képernyőn, megnöveljük a számlálónk értékét, kiírjuk, hogy elkészült a kép, lementjük a harddiskre és várunk egy másodpercet. 

Ezután az opencv csomag segítségével leellenőrízzük, hogy megnyomták-e a *q* billentyűt: ```if cv2.waitKey(100) & 0xFF == ord('q'):```, és ha igen, akkor kilépünk a ciklusból. A ```cv2.waitKey(100)``` parancs 100 milimásodpercet vár a továbbhaladás előtt és érzékeli, ha közben megnyomtak, egy billentyűt. A várakozási időt lehet módosítani. A kilépést követően, bezárjuk a kapcsolatot a kamerával, bezárjuk az ablakokat és a képekből videót készítünk.

```stop_motion.py```:

In [51]:
import cv2, time, subprocess
from gpiozero import Button

cap = cv2.VideoCapture(0)  # cap = cv2.VideoCapture('/dev/video0')
button = Button(2)
nframe = 0

while True:
    button.wait_for_press()
    ret, frame = cap.read()
    cv2.imshow('frame', frame)
    nframe = nframe + 1
    print(f'snaphot {nframe:03d} done!')
    cv2.imwrite(f'animation/frame{nframe:03d}.jpg', frame)
    time.sleep(1)
    
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cap.release()
# Bezarunk minden ablakot, amit a program megnyitott
cv2.destroyAllWindows()

subprocess.call('ffmpeg -r 1 -i animation/frame%03d.jpg -qscale 1 animation.mp4')

### Filmkészítés laptoppal

Ha laptopot használunk (és pl. annak beépített kameráját) és billentyűzetét a műveletek elvégzésére, akkor minimálisan módosítani kell a kódot. 

Itt a képet folyamatosan megjelenítjük, de csak akkor készítünk képet, ha a *p* billentyű lenyomását érzékeltük, ```if cv2.waitKey(100) & 0xFF == ord('p'):```. A többi része a kódnak lényegében ugyanaz.

```stop_motion.py```:

In [64]:
import cv2, time, subprocess

cap = cv2.VideoCapture(0)   # cap = cv2.VideoCapture('/dev/video0')
nframe = 0

while True:
    ret, frame = cap.read()
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(100) & 0xFF == ord('p'):
        nframe = nframe + 1
        print(f'snaphot {nframe:03d} done!')
        cv2.imwrite(f'animation/frame{nframe:03d}.jpg', frame)
    
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break
    time.sleep(0.2)

cap.release()
# Bezarunk minden ablakot, amit a program megnyitott
cv2.destroyAllWindows()

subprocess.call('ffmpeg -r 1 -i animation/frame%03d.jpg -qscale 1 animation.mp4')

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```stop_motion.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```stop_motion.py```-t elmentettük. Ott begépelve a ```python stop_motion.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a program elindításával megjelenik a kamera által látott kép, és a gomb lenyomásával elkészíthetjük a fényképeket ha készen állunk rá. Ha nem akarunk több képet készíteni, megnyomhatjuk a *q* billentyűt és kilépünk a programból és egyben elkészül a videó is.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Módosítsuk a kódot úgy, hogy ne a *q* billentyű lenyomása, hanem egy újabb nyomógomb jelentse a műveletek befejezését.
* Változtasd meg a frame rate értékét a filmben és nézd meg, hogyan változik a filmed.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html

2) https://ffmpeg.org/ffmpeg.html

3) https://gpiozero.readthedocs.io/en/stable/api_input.html?highlight=button#gpiozero.Button

4) https://docs.python.org/3/library/asyncio-subprocess.html

5) https://www.tutorialspoint.com/python/time_sleep.htm